In [1]:
pip install tensorflow-io

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import tensorflow as tf
import numpy as np
import wave
from pathlib import Path
from dataclasses import dataclass
import tensorflow_io as tfio


In [4]:
tf.compat.v1.enable_eager_execution()
print("Eager execution enabled:", tf.executing_eagerly())

Eager execution enabled: True


In [5]:
def check_sample_rate(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.wav'):
                filepath = os.path.join(root, file)
                try:
                    with wave.open(filepath, 'rb') as wav_file:
                        sample_rate = wav_file.getframerate()
                        print(f"File: {filepath}, Sample Rate: {sample_rate} Hz")
                except wave.Error as e:
                    print(f"Error reading {filepath}: {e}")

data_dir = 'dataset_small'  # Replace with your data directory
check_sample_rate(data_dir)

File: dataset_small/no/b43c8f2f_nohash_1.wav, Sample Rate: 16000 Hz
File: dataset_small/no/c120e80e_nohash_2.wav, Sample Rate: 16000 Hz
File: dataset_small/no/3a69f765_nohash_4.wav, Sample Rate: 16000 Hz
File: dataset_small/no/47d01978_nohash_1.wav, Sample Rate: 16000 Hz
File: dataset_small/no/ee202358_nohash_1.wav, Sample Rate: 16000 Hz
File: dataset_small/no/1e4064b8_nohash_0.wav, Sample Rate: 16000 Hz
File: dataset_small/no/530306f5_nohash_0.wav, Sample Rate: 16000 Hz
File: dataset_small/no/0cb74144_nohash_4.wav, Sample Rate: 16000 Hz
File: dataset_small/no/e5dadd24_nohash_0.wav, Sample Rate: 16000 Hz
File: dataset_small/no/6f1ffef6_nohash_2.wav, Sample Rate: 16000 Hz
File: dataset_small/no/ce0cb033_nohash_1.wav, Sample Rate: 16000 Hz
File: dataset_small/no/0132a06d_nohash_4.wav, Sample Rate: 16000 Hz
File: dataset_small/no/36050ef3_nohash_1.wav, Sample Rate: 16000 Hz
File: dataset_small/no/0f46028a_nohash_3.wav, Sample Rate: 16000 Hz
File: dataset_small/no/888a0c49_nohash_0.wav, Sa

File: dataset_small/no/525eaa62_nohash_4.wav, Sample Rate: 16000 Hz
File: dataset_small/no/18c54a68_nohash_0.wav, Sample Rate: 16000 Hz
File: dataset_small/no/5b09db89_nohash_1.wav, Sample Rate: 16000 Hz
File: dataset_small/no/b69002d4_nohash_3.wav, Sample Rate: 16000 Hz
File: dataset_small/no/d962e5ac_nohash_1.wav, Sample Rate: 16000 Hz
File: dataset_small/no/8134f43f_nohash_0.wav, Sample Rate: 16000 Hz
File: dataset_small/no/0d85a428_nohash_1.wav, Sample Rate: 16000 Hz
File: dataset_small/no/32ad5b65_nohash_2.wav, Sample Rate: 16000 Hz
File: dataset_small/no/3efef882_nohash_1.wav, Sample Rate: 16000 Hz
File: dataset_small/no/39afb88e_nohash_0.wav, Sample Rate: 16000 Hz
File: dataset_small/no/3a3ee7ed_nohash_2.wav, Sample Rate: 16000 Hz
File: dataset_small/no/a5b24175_nohash_3.wav, Sample Rate: 16000 Hz
File: dataset_small/no/876c84d6_nohash_2.wav, Sample Rate: 16000 Hz
File: dataset_small/no/333d7ddb_nohash_0.wav, Sample Rate: 16000 Hz
File: dataset_small/no/0717b9f6_nohash_2.wav, Sa

File: dataset_small/yes/b43c8f2f_nohash_1.wav, Sample Rate: 16000 Hz
File: dataset_small/yes/c120e80e_nohash_2.wav, Sample Rate: 16000 Hz
File: dataset_small/yes/3a69f765_nohash_4.wav, Sample Rate: 16000 Hz
File: dataset_small/yes/ee202358_nohash_1.wav, Sample Rate: 16000 Hz
File: dataset_small/yes/211ccd2e_nohash_0.wav, Sample Rate: 16000 Hz
File: dataset_small/yes/530306f5_nohash_0.wav, Sample Rate: 16000 Hz
File: dataset_small/yes/ce0cb033_nohash_1.wav, Sample Rate: 16000 Hz
File: dataset_small/yes/9d7036f9_nohash_2.wav, Sample Rate: 16000 Hz
File: dataset_small/yes/0132a06d_nohash_4.wav, Sample Rate: 16000 Hz
File: dataset_small/yes/36050ef3_nohash_1.wav, Sample Rate: 16000 Hz
File: dataset_small/yes/0f46028a_nohash_3.wav, Sample Rate: 16000 Hz
File: dataset_small/yes/8eb4a1bf_nohash_4.wav, Sample Rate: 16000 Hz
File: dataset_small/yes/df1d5024_nohash_0.wav, Sample Rate: 16000 Hz
File: dataset_small/yes/888a0c49_nohash_0.wav, Sample Rate: 16000 Hz
File: dataset_small/yes/df1d5024_n

File: dataset_small/yes/baf01c1f_nohash_1.wav, Sample Rate: 16000 Hz
File: dataset_small/yes/be91a00a_nohash_0.wav, Sample Rate: 16000 Hz
File: dataset_small/yes/b737ee80_nohash_0.wav, Sample Rate: 16000 Hz
File: dataset_small/yes/190821dc_nohash_0.wav, Sample Rate: 16000 Hz
File: dataset_small/yes/cd85758f_nohash_3.wav, Sample Rate: 16000 Hz
File: dataset_small/yes/c1e0e8e3_nohash_2.wav, Sample Rate: 16000 Hz
File: dataset_small/yes/f8ba7c0e_nohash_1.wav, Sample Rate: 16000 Hz
File: dataset_small/yes/1a892463_nohash_0.wav, Sample Rate: 16000 Hz
File: dataset_small/yes/ffd2ba2f_nohash_0.wav, Sample Rate: 16000 Hz
File: dataset_small/yes/48bfde8e_nohash_1.wav, Sample Rate: 16000 Hz
File: dataset_small/yes/87070229_nohash_1.wav, Sample Rate: 16000 Hz
File: dataset_small/yes/a1cff772_nohash_2.wav, Sample Rate: 16000 Hz
File: dataset_small/yes/a0f93943_nohash_1.wav, Sample Rate: 16000 Hz
File: dataset_small/yes/06f6c194_nohash_1.wav, Sample Rate: 16000 Hz
File: dataset_small/yes/1365dd89_n

In [6]:
@dataclass
class FeatureParams:
    sample_rate: int = 16000  # Audio sample rate
    window_size_ms: float = 30.0  # Window size in milliseconds
    window_stride_ms: float = 20.0  # Window stride in milliseconds
    num_mel_bins: int = 40  # Number of Mel bins
    lower_frequency: float = 125.0  # Lower frequency limit in Hz
    upper_frequency: float = 7500.0  # Upper frequency limit in Hz
    fft_length: int = None  # FFT length
    clip_duration_ms: float = 1000.0  # Duration to clip/pad audio in milliseconds

    def __post_init__(self):
        self.window_length_samples = int(self.sample_rate * self.window_size_ms / 1000)
        self.window_step_samples = int(self.sample_rate * self.window_stride_ms / 1000)
        if self.fft_length is None:
            self.fft_length = 2 ** int(np.ceil(np.log2(self.window_length_samples)))
        self.desired_samples = int(self.sample_rate * self.clip_duration_ms / 1000)


In [7]:
class AudioProcessor:
    def __init__(self, params: FeatureParams):
        self.params = params

    def load_wav_file(self, filename: str) -> tf.Tensor:
        audio_binary = tf.io.read_file(filename)
        waveform, sample_rate = tf.audio.decode_wav(audio_binary, desired_channels=1)
        waveform = tf.squeeze(waveform, axis=-1)
        sample_rate = tf.cast(sample_rate, tf.int32)
        #print(sample_rate.shape)
        #print(sample_rate)
        #if sample_rate != self.params.sample_rate:
        #    print(f"Warning: Sample rate mismatch for {filename}. Resampling from {sample_rate} Hz to {self.params.sample_rate} Hz.")
        #    # Resample the waveform
        #    waveform = tfio.audio.resample(waveform, rate_in=tf.cast(sample_rate, tf.int64), rate_out=self.params.sample_rate)
        # Trim or pad waveform to desired_samples
        waveform = waveform[:self.params.desired_samples]
        zero_padding = tf.zeros(
            [self.params.desired_samples - tf.shape(waveform)[0]], dtype=tf.float32)
        waveform = tf.concat([waveform, zero_padding], 0)
        return waveform

    def process_waveform(self, waveform: tf.Tensor) -> tf.Tensor:
        # Generate frames
        frames = tf.signal.frame(
            waveform,
            self.params.window_length_samples,
            self.params.window_step_samples,
            pad_end=True
        )
        # Apply Hann window
        window = tf.signal.hann_window(self.params.window_length_samples)
        windowed_frames = frames * window
        # Compute FFT
        fft = tf.signal.rfft(windowed_frames, [self.params.fft_length])
        power_spectrum = tf.abs(fft) ** 2
        # Apply Mel filter bank
        num_spectrogram_bins = self.params.fft_length // 2 + 1
        linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(
            self.params.num_mel_bins,
            num_spectrogram_bins,
            self.params.sample_rate,
            self.params.lower_frequency,
            self.params.upper_frequency
        )
        mel_spectrogram = tf.tensordot(power_spectrum, linear_to_mel_weight_matrix, 1)
        mel_spectrogram.set_shape(power_spectrum.shape[:-1].concatenate(
            [self.params.num_mel_bins]))
        # Compute log-mel spectrogram
        log_mel_spectrogram = tf.math.log(mel_spectrogram + 1e-6)

        # Ensure fixed number of frames
        expected_num_frames = 1 + (self.params.desired_samples - self.params.window_length_samples) // self.params.window_step_samples
        num_frames = tf.shape(log_mel_spectrogram)[0]
        num_padding_frames = expected_num_frames - num_frames

        # Pad or truncate to expected_num_frames without using Python conditionals
        num_padding_frames = tf.maximum(num_padding_frames, 0)
        log_mel_spectrogram = tf.pad(log_mel_spectrogram, [[0, num_padding_frames], [0, 0]], "CONSTANT")
        log_mel_spectrogram = log_mel_spectrogram[:expected_num_frames, :]

        return log_mel_spectrogram


In [8]:
def process_dataset(data_dir: str, params: FeatureParams):
    processor = AudioProcessor(params)
    labels = sorted(os.listdir(data_dir))
    label_to_index = {label: index for index, label in enumerate(labels)}
    data = []
    data_labels = []
    for label in labels:
        label_dir = os.path.join(data_dir, label)
        if not os.path.isdir(label_dir):
            continue
        print(f"Processing label: {label}")
        files = os.listdir(label_dir)
        for filename in files:
            if filename.endswith('.wav'):
                filepath = os.path.join(label_dir, filename)
                waveform = processor.load_wav_file(filepath)
                if waveform is None:
                    continue
                features = processor.process_waveform(waveform)
                features_np = features.numpy()
                data.append(features_np)
                data_labels.append(label_to_index[label])
    return np.array(data), np.array(data_labels)

In [9]:
def save_processed_data(features: np.ndarray, labels: np.ndarray, output_dir: str):
    os.makedirs(output_dir, exist_ok=True)
    np.save(os.path.join(output_dir, 'features.npy'), features)
    np.save(os.path.join(output_dir, 'labels.npy'), labels)
    print(f"Saved features to {os.path.join(output_dir, 'features.npy')}")
    print(f"Saved labels to {os.path.join(output_dir, 'labels.npy')}")

In [10]:
data_dir = 'dataset_small'  # Replace with your data directory
output_dir = 'processed_dataset_small'  # Replace with your desired output directory

# Initialize parameters
params = FeatureParams(
    sample_rate=16000,
    window_size_ms=30.0,
    window_stride_ms=20.0,
    num_mel_bins=40,
    lower_frequency=125.0,
    upper_frequency=7500.0,
    clip_duration_ms=1000.0
)

# Process dataset
features, labels = process_dataset(data_dir, params)

# Save processed data
save_processed_data(features, labels, output_dir)

Processing label: no
Processing label: yes
Saved features to processed_dataset_small/features.npy
Saved labels to processed_dataset_small/labels.npy


In [ ]:
print("Eager execution enabled:", tf.executing_eagerly())